# Import Packages

In [ ]:
import numpy as np
import pandas as pd
from scipy.io import loadmat
import tensorflow.keras as keras
from sklearn.preprocessing import MinMaxScaler

# Dataset Path Setting
make sure the paths are correct.


In [ ]:
class Datasets:
    MFPT_BASE = "/media/sda1/dataset/MFPT/MFPT Fault Data Sets"
    MFPT_DATA = "1 - Three Baseline Conditions"
    MFPT_MAT = "baseline_1.mat"
    MFPT_ABNORMAL_DATA = "2 - Three Outer Race Fault Conditions"
    MFPT_ABNORMAL_MAT = "OuterRaceFault_1.mat"

# Data Preprocessing
Preprocessing datas before the training
Simply change dataset shape in this example.

In [ ]:
class Preprocessing:

    @staticmethod
    def create_dataset(data, window_size, preprocess=False):
        org_shape = data.shape
        truncated = data.size - (int(data.size/window_size) * window_size)
        data = data[:data.size - truncated]
        
        if preprocess:
            minmax = MinMaxScaler()
            data = minmax.fit_transform(data)
        dataset = np.empty((int(data.size/window_size), window_size, 1))
        j = 0
        for i in [n*window_size for n in range(int(data.size/window_size))]:
            dataset[j] = data[i:i+window_size]
            j = j + 1
            
        print(f"Dataset shape changed {org_shape} -> {dataset.shape}.")
        print(f"Truncated {truncated} values ")
        #print(f"{data[data.size-100:data.size-50]}, {dataset[-2]}")
        #print(f"{data[data.size-50:data.size]}, {dataset[-1]}")
        return dataset
    
    @staticmethod
    def create_overlap_dataset(data, window_size, preprocess=False):
        org_shape = data.shape
        if preprocess:
            minmax = MinMaxScaler()
            data = minmax.fit_transform(data)
        dataset = np.empty((data.size-window_size+1, window_size, 1))
        j = 0
        for i in range(data.size-window_size+1):
            dataset[j] = data[i:i+window_size]
            j = j + 1
        print(f"Dataset shape changed {org_shape} -> {dataset.shape}")
        #print(f"{data[data.size-100:data.size-50]}, {dataset[-2]}")
        #print(f"{data[data.size-50:data.size]}, {dataset[-1]}")
        return dataset

# Configuring Model

In [ ]:
class Model:
    #https://machinelearningmastery.com/lstm-autoencoders/
    def __init__(self, x_train):
        self.x_train = x_train
    
    def lstm_autoencoder(self, input_size, epochs):
        output_size = input_size
        model = keras.models.Sequential([
            keras.layers.RNN(keras.layers.LSTMCell(128), input_shape=[input_size, 1]),
            keras.layers.RepeatVector(output_size),
            keras.layers.RNN(keras.layers.LSTMCell(128), return_sequences=True),
            keras.layers.TimeDistributed(keras.layers.Dense(1))
        ])
        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(self.x_train, self.x_train, epochs=epochs)
        return model
    
    def rnn_autoencoder(self, input_size, epochs):
        output_size = input_size
        model = keras.models.Sequential([
            keras.layers.SimpleRNN(128, input_shape=[input_size, 1]),
            keras.layers.RepeatVector(output_size),
            keras.layers.SimpleRNN(128, return_sequences=True),
            keras.layers.TimeDistributed(keras.layers.Dense(1))
        ])
        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(self.x_train, self.x_train, epochs=epochs)
        result = model.predict(self.x_train)
        return result

# Load Dataset

In [ ]:
    import os
    path = os.path.join(Datasets.MFPT_BASE, Datasets.MFPT_DATA, Datasets.MFPT_MAT)
    
    mat = loadmat(path)
    keys = mat['bearing'].dtype.fields
    mat_dict = {
        list(keys.keys())[0] : mat['bearing'][0][0][0],
        list(keys.keys())[1] : mat['bearing'][0][0][1],
        list(keys.keys())[2] : mat['bearing'][0][0][2],
        list(keys.keys())[3] : mat['bearing'][0][0][3]
    }
    dataset = Preprocessing.create_dataset(mat_dict['gs'], 50)

# LSTM AutoEncoder

# Train Model & Save Result

In [ ]:
import datetime
now = datetime.datetime.now().strftime('%Y_%m_%d_%H:%M')

epochs = 20
input_size = 50

model = Model(dataset)
lstm_ae_model = model.lstm_autoencoder(input_size, epochs=epochs)

In [ ]:
result = lstm_ae_model.predict(dataset)

In [ ]:
# Save
import os

DIR = "./results"

if not os.path.exists(DIR):
    os.makedirs(DIR)
    
lstm_ae_model.save(f'{DIR}/MFPT_lstmAE_{epochs}_{input_size}_{now}')
np.save(f'{DIR}/MFPT_lstmAE_{epochs}_{input_size}_{now}/MFPT_lstmAE_{epochs}_{input_size}_{now}.npy', result) #Save Results to npy file


# Load Result

In [ ]:
result = np.load(f'{DIR}/MFPT_lstmAE_{epochs}_{input_size}_{now}/MFPT_lstmAE_{epochs}_{input_size}_{now}.npy')
#print(f"{dataset}, {result}")

In [ ]:
import matplotlib.pyplot as plt

RANGE = dataset.size #dataset.size

dataset = np.reshape(dataset, dataset.size)
result = np.reshape(result, result.size)

fig, axs = plt.subplots(figsize = (20,10)) # This is your answer to resize the figure
# The below will help you expand on your question and resize individual elements within your figure. Experiement with the below parameters.
axs.set_title(f"LSTM Auto Encoder : epoch[{epochs}], input_size[{input_size}]", fontsize = 17.5)
axs.tick_params(axis = 'x', labelsize = 14)
axs.set_xlabel('Timestamp', size = 15)
axs.tick_params(axis = 'y', labelsize =14)
axs.set_ylabel('Value', size = 15)

plt.plot(dataset[:RANGE], 'r', linewidth=1, label="Dataset")
plt.plot(result[:RANGE], 'g', linewidth=1, label="Predicted")
plt.legend()
plt.savefig(f'{DIR}/MFPT_lstmAE_{epochs}_{input_size}_{now}/MFPT_lstmAE_{epochs}_{input_size}_{now}.png')

# Compare Results

In [ ]:
DIR = "./results"
FILE_NAME = "MFPT_lstmAE_10_50_2022_01_25_15:54.npy"
result1 = np.load(f'{DIR}/{FILE_NAME}')
FILE_NAME = "MFPT_lstmAE_20_50_2022_02_03_20:11.npy"
result2 = np.load(f'{DIR}/{FILE_NAME}')
FILE_NAME = "MFPT_lstmAE_5_50_2022_01_25_15:34.npy"
result3 = np.load(f'{DIR}/{FILE_NAME}')

In [ ]:
import matplotlib.pyplot as plt

RANGE = 300 #dataset.size

dataset = np.reshape(dataset, dataset.size)
result1 = np.reshape(result1, result1.size)
result2 = np.reshape(result2, result2.size)
result3 = np.reshape(result3, result3.size)

fig, axs = plt.subplots(figsize = (20,10)) # This is your answer to resize the figure
# The below will help you expand on your question and resize individual elements within your figure. Experiement with the below parameters.
axs.set_title("Data", fontsize = 17.5)
axs.tick_params(axis = 'x', labelsize = 14)
axs.set_xlabel('Timestamp', size = 15)
axs.tick_params(axis = 'y', labelsize =14)
axs.set_ylabel('Value', size = 15)

plt.plot(dataset[:RANGE], 'r', linewidth=1, label="Dataset")
#plt.plot(result1[:RANGE], 'g', linewidth=1, label="Epoch 10")
#plt.plot(result2[:RANGE], 'c', linewidth=1, label="Epoch 20")
plt.plot(result3[:RANGE], 'y', linewidth=1, label="Epoch 5")

plt.legend()
plt.savefig(f"{DIR}/MFPT_lstmAE_5_50_2022_01_24_20:29.png")